In [1]:
# import packages
import matplotlib.pyplot as plt
import pandas as pd

"""
IMPORTANT:
For this program to work the researcher needs to put both csv files
cycle1.csv and cycle2.csv in a folder called Observations. Run
DownloadMongoDB.ipynb to get the files. You will have to manually
put these new files in the Observations folder which will be located
in the same folder as this file.
"""

# import data from csv file into a pandas df
cycle1_df = pd.read_csv("Observations/cycle1.csv", low_memory=False)
cycle2_df = pd.read_csv("Observations/cycle2.csv", low_memory=False)

# TODO: plot the data
print(cycle1_df.head())
print(cycle2_df.head())


                        _id Original_Object_ID           MJD  Magnitude  \
0  67cc99fbb61a68894114bf37               1000  58596.438921 -10.173042   
1  67cc99fbb61a68894114bf38               1000  58596.459754 -10.097651   
2  67cc99fbb61a68894114bf39               1000  58596.480587  -9.936967   
3  67cc99fbb61a68894114bf3a               1000  58596.563920  -9.899730   
4  67cc99fbb61a68894114bf3b               1000  58596.584754 -10.061810   

   Magnitude_Error  
0         0.075739  
1         0.081009  
2         0.076457  
3         0.072220  
4         0.056424  
                        _id Object_ID           MJD  Magnitude  \
0  681e7faa4281eb4c05317e35         6  58953.873393 -20.574373   
1  681e7faa4281eb4c05317e36         6  58953.894226 -21.098431   
2  681e7faa4281eb4c05317e37         6  58953.915059 -21.065185   
3  681e7faa4281eb4c05317e38         6  58953.935891 -21.066721   
4  681e7faa4281eb4c05317e39         6  58953.956724 -21.086373   

   Magnitude_Error  Sector